In [ ]:
# Python Week 16

## This exercise is designed for us to learn pytorch through struggles and debugging. 
## The code is not complete. Today's task is to make this work.

We are trying to convert our previous pytorch classification example to pytorch autoencoding (denoising) example. Enjoy!

Please read the following tutorial 
https://pytorch.org/tutorials.

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

In [ ]:
trainset = datasets.MNIST('.\\', download=True, train=True, transform=transform);
valset   = datasets.MNIST('.\\', download=True, train=False, transform=transform);


trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
valloader   = torch.utils.data.DataLoader(valset, batch_size = 64, shuffle = True)

In [ ]:
#help(trainloader)

In [ ]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

In [ ]:
def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images +1 ):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')

In [ ]:
print(images.shape)
print(labels.shape)

In [ ]:
## This example showed up in our week 11 session.
## We do not use this but this is given as a reference.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        print(x.shape)
        x = F.max_pool2d(x, 2)
        print(x.shape)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        print(x.shape)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
     
net = Net()
net.zero_grad()
print(net)

## test if the network works
input = torch.randn(1, 1, 28, 28)

## this step will print out 
out = net(input)
out.backward(torch.randn(1,10))

In [ ]:
## this is the class for autoencoding.

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
        nn.Conv2d(1, 16, 3, 1),
        nn.ReLU(),
        nn.Dropout2d(0.25),
        nn.Conv2d(16, 32, 3, 1),
        nn.ReLU(),
        nn.Dropout2d(0.25),
        nn.Conv2d(32, 64, 7)
        )
        
        
        self.decoder = nn.Sequential(
        nn.ConvTranspose2d(64, 32, 7),
        nn.ReLU(),
        nn.ConvTranspose2d(32, 16, 3),
        nn.ReLU(),
        nn.ConvTranspose2d(16, 1, 3, 1),
        nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
net = AutoEncoder()
net.zero_grad()
print(net)

## test if the network works
input = torch.randn(1, 1, 28, 28)

## this step will print out 
out = net(input)
out.backward(torch.randn(1,1,28,28))    

print(out.shape)

In [ ]:

## pay attention to the difference from our previous week 10 example
## 
criterion = torch.nn.MSELoss(reduction = 'sum');

optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)


In [ ]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        optimizer.zero_grad()
        
        # forward + backward + optimize
        
        outputs = net(inputs)
        
        
        #print(outputs.shape)
        #print(inputs.shape)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i+1, running_loss/2000))
            running_loss = 0.0
            
print('Finished Training')


In [ ]:
vdataiter = iter(valloader)
vimages, vlabels = next(vdataiter)

classes = ('0', '1','2', '3', '4','5', '6','7', '8','9')

imshow(torchvision.utils.make_grid(vimages))

num_show = 10

print('First 10 images')
print('GroundTruth: ',  ''.join('%5s'%classes[vlabels[j]] for j in range(num_show)))

outputs = net(vimages)

_, predicted  = torch.max(outputs, 1)
print('Predicted:   ', ''.join('%5s'%classes[predicted[j]] for j in range(num_show)))